In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading AI model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Simple example
text = "The moon was shining bright at night, casting a soft silver glow across the quiet streets and the empty sky."
print(f"\nStarting text: '{text}'")
print("\nGenerating word by word...\n")

# Generate 5 words, one at a time
current_text = text
steps = 30
k = 10
greedy = False
for step in range(steps):
    print(f"--- Step {step + 1} ---")
    print(f"Current: '{current_text}'")

    # Encode current text
    input_ids = tokenizer.encode(current_text, return_tensors="pt")

    # Get predictions
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits

    # Get the predictions for the NEXT token
    next_token_logits = predictions[0, -1, :]
    next_token_probs = torch.softmax(next_token_logits, dim=0)

    # Get top k predictions
    top_probs, top_indices = torch.topk(next_token_probs, k)

    print("Top", k, "next word predictions:")
    for i, (prob, idx) in enumerate(zip(top_probs, top_indices)):
        word = tokenizer.decode([idx])
        print(f"  {i+1}. '{word}' ({prob.item()*100:.1f}% confident)")

    # part 5
    if(greedy):
      next_token_id = top_indices[0]
    else:
      next_token_id = top_indices[random.randint(0, k - 1)]

    # Use the most likely word
    next_word = tokenizer.decode([next_token_id])
    current_text += next_word

    print(f"✓ Chosen: '{next_word}'")
    print(f"New text: '{current_text}'\n")

print(f"\nFinal generated text: '{current_text}'")


Starting text: 'The weather today is '

Generating word by word...

--- Step 1 ---
Current: 'The weather today is '
Top 5 next word predictions:
  1. 'iced' (45.6% confident)
  2. ' ' (18.6% confident)
  3. '!!!' (2.5% confident)
  4. '�' (2.0% confident)
  5. '????' (1.8% confident)
✓ Chosen: 'iced'
New text: 'The weather today is iced'

--- Step 2 ---
Current: 'The weather today is iced'
Top 5 next word predictions:
  1. ' up' (13.1% confident)
  2. ' with' (11.3% confident)
  3. ' and' (8.4% confident)
  4. '.' (8.0% confident)
  5. ',' (8.0% confident)
✓ Chosen: ' up'
New text: 'The weather today is iced up'

--- Step 3 ---
Current: 'The weather today is iced up'
Top 5 next word predictions:
  1. ',' (15.7% confident)
  2. ' and' (14.9% confident)
  3. '.' (10.3% confident)
  4. ' by' (6.9% confident)
  5. ' to' (4.9% confident)
✓ Chosen: ','
New text: 'The weather today is iced up,'

--- Step 4 ---
Current: 'The weather today is iced up,'
Top 5 next word predictions:
  1. ' and' 